In [1]:
import nltk
from collections import Counter
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.tokenize import MWETokenizer
from nltk import regexp_tokenize
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

import string
from collections import Counter

import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
!pip install textstat
import textstat
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import numpy as np

[nltk_data] Downloading package stopwords to /Users/water/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /Users/water/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /Users/water/nltk_data...


  Obtaining dependency information for textstat from https://files.pythonhosted.org/packages/d8/33/302083f47386d651e4b42923f5206eeb9ee0545ea94bb506324d05fd2274/textstat-0.7.3-py3-none-any.whl.metadata
  Obtaining dependency information for pyphen from https://files.pythonhosted.org/packages/a1/5d/a545d71eba7e031ffd4c670b18b7c7c8b128d0fd0ca79914bd40ab98a456/pyphen-0.15.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 4.1 MB/s eta 0:00:00a 0:00:01


In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir(f'/content/drive/MyDrive/Advanced Topics NLP and RL/DURF Data')

Mounted at /content/drive


In [ ]:
# Check Data
stocksInvesting1 = pd.read_csv("example_posts (1).csv")
stocksInvesting2 = pd.read_csv("investingstocks.csv")

stocksInvesting1.head(5)
stocksInvesting1.info()
stocksInvesting2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   subreddit_name       300 non-null    object 
 1   post_created_utc     300 non-null    float64
 2   id                   300 non-null    object 
 3   selftext             300 non-null    object 
 4   is_original_content  300 non-null    bool   
 5   is_self              300 non-null    bool   
 6   link_flair_text      82 non-null     object 
 7   locked               300 non-null    bool   
 8   num_comments         300 non-null    int64  
 9   over_18              300 non-null    bool   
 10  score                300 non-null    int64  
 11  spoiler              300 non-null    bool   
 12  stickied             300 non-null    bool   
 13  title                300 non-null    object 
 14  upvote_ratio         300 non-null    float64
 15  url                  300 non-null    obj

In [ ]:
# Combine 2 datasets together
stocksInvestingMerged = pd.concat([stocksInvesting1, stocksInvesting2])
stocksInvestingMerged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2181 entries, 0 to 1880
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   subreddit_name       2181 non-null   object 
 1   post_created_utc     2181 non-null   float64
 2   id                   2181 non-null   object 
 3   selftext             2180 non-null   object 
 4   is_original_content  2181 non-null   bool   
 5   is_self              2181 non-null   bool   
 6   link_flair_text      605 non-null    object 
 7   locked               2181 non-null   bool   
 8   num_comments         2181 non-null   int64  
 9   over_18              2181 non-null   bool   
 10  score                2181 non-null   int64  
 11  spoiler              2181 non-null   bool   
 12  stickied             2181 non-null   bool   
 13  title                2181 non-null   object 
 14  upvote_ratio         2181 non-null   float64
 15  url                  2181 non-null   o

In [ ]:
# Check if duplicate Data
stocksInvestingMerged["id"].is_unique

False

In [ ]:
# Remove Duplicate Data
stocksInvestingMerged.drop_duplicates(subset='id', inplace = True)
stocksInvestingMerged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1883 entries, 0 to 1880
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   subreddit_name       1883 non-null   object 
 1   post_created_utc     1883 non-null   float64
 2   id                   1883 non-null   object 
 3   selftext             1882 non-null   object 
 4   is_original_content  1883 non-null   bool   
 5   is_self              1883 non-null   bool   
 6   link_flair_text      524 non-null    object 
 7   locked               1883 non-null   bool   
 8   num_comments         1883 non-null   int64  
 9   over_18              1883 non-null   bool   
 10  score                1883 non-null   int64  
 11  spoiler              1883 non-null   bool   
 12  stickied             1883 non-null   bool   
 13  title                1883 non-null   object 
 14  upvote_ratio         1883 non-null   float64
 15  url                  1883 non-null   o

In [ ]:
# StockMaster contain post that only relates to "Stocks" posts
stocksMaster = stocksInvestingMerged[stocksInvestingMerged["subreddit_name"] == "stocks"]
# investingMaster contain post that only relates to "Investing" posts.
wsbMaster = pd.read_csv("testdata.csv")
investingMaster = stocksInvestingMerged[stocksInvestingMerged["subreddit_name"] == "investing"]
SMmaster = pd.read_csv("stockmarket.csv")

In [ ]:
wsbMaster

,Post ID,Title,Url,Author,Score,Publish Date,Selftext,Flair
0,eifu11,DD: AZN / MRK Got Approval For New pancreatic ...,https://www.reddit.com/r/wallstreetbets/commen...,micholas_norse,1,2020-01-01 07:51:58,AZN - Buy $50 call 1/3/20\nMRK - Buy $91 call ...,"[{'e': 'text', 't': 'DD'}]"
1,eiop2z,DD on pot stocks,https://www.reddit.com/r/wallstreetbets/commen...,Black_Ink,1,2020-01-01 22:15:00,"First day of legal weed in Illinois, passed by...",[]
2,ej1m7j,Looking for DD on Australian firms or internat...,https://www.reddit.com/r/wallstreetbets/commen...,SFWonlyInvest,5,2020-01-02 18:00:00,"So the fires, how do we long this. Or to make ...",[]
3,ej9pgj,Enough DD to me. 😎,https://i.redd.it/yqth6t4vjh841.jpg,Frankbux47,1,2020-01-03 03:47:55,NaN,"[{'e': 'text', 't': 'Shitpost'}]"
4,ejadxp,Signal to short market. Solid DD,https://www.reddit.com/r/wallstreetbets/commen...,macrodog,11,2020-01-03 04:45:16,I think I finally cracked the code when to sho...,"[{'e': 'text', 't': 'Shitpost'}]"
...,...,...,...,...,...,...,...,...
3769,k5v4jd,the only crayon DD you guys need,https://i.redd.it/f97xiwbldy261.jpg,Youkiame,1,2020-12-03 10:56:15,NaN,"[{'e': 'text', 't': 'Meme'}]"
3770,k5xzb7,What is your go to source for doing DD on a co...,https://www.reddit.com/r/wallstreetbets/commen...,figgepop,1,2020-12-03 14:17:04,"As the title says, where do you go to do an in...","[{'e': 'text', 't': 'Discussion'}]"
3771,k5y7nk,Not sure why this sundials are supposed to boo...,https://i.redd.it/ayc8gtdvfz261.jpg,xNeon14,1,2020-12-03 14:30:27,NaN,"[{'e': 'text', 't': 'Shitpost'}]"
3772,k5zljj,DD is overrated,https://i.redd.it/7yr3yqansz261.jpg,jimmykim9001,1,2020-12-03 15:45:11,NaN,"[{'e': 'text', 't': 'Meme'}]"


In [ ]:
# Filtering flair, getting rid of the useless ones
dropFlair1Idx = (wsbMaster.loc[wsbMaster['Flair'].str.contains("Shitpost", case=False)]).index
dropFlair2Idx = (wsbMaster.loc[wsbMaster['Flair'].str.contains("Meme", case=False)]).index

wsbMaster2 = wsbMaster.drop(index = list(dropFlair1Idx))
wsbMaster2 = wsbMaster2.drop(index = list(dropFlair2Idx))

wsbMaster2.reset_index(drop=True)

,Post ID,Title,Url,Author,Score,Publish Date,Selftext,Flair
0,eifu11,DD: AZN / MRK Got Approval For New pancreatic ...,https://www.reddit.com/r/wallstreetbets/commen...,micholas_norse,1,2020-01-01 07:51:58,AZN - Buy $50 call 1/3/20\nMRK - Buy $91 call ...,"[{'e': 'text', 't': 'DD'}]"
1,eiop2z,DD on pot stocks,https://www.reddit.com/r/wallstreetbets/commen...,Black_Ink,1,2020-01-01 22:15:00,"First day of legal weed in Illinois, passed by...",[]
2,ej1m7j,Looking for DD on Australian firms or internat...,https://www.reddit.com/r/wallstreetbets/commen...,SFWonlyInvest,5,2020-01-02 18:00:00,"So the fires, how do we long this. Or to make ...",[]
3,eje19e,world war DD,https://www.reddit.com/r/wallstreetbets/commen...,seaisthememes,5,2020-01-03 11:10:14,[removed],"[{'e': 'text', 't': 'DD'}]"
4,ejn7w2,SHORT SQ TO 61 DD INSIDE,https://i.imgur.com/VLyRvXP.jpg,DeliciouslyUnaware,16,2020-01-03 22:50:04,NaN,"[{'e': 'text', 't': 'Technicals'}]"
...,...,...,...,...,...,...,...,...
3069,k5rxle,"$JKS short DD post, why this will be the next ...",https://www.reddit.com/r/wallstreetbets/commen...,threefourpizza,1,2020-12-03 06:21:33,[removed],"[{'e': 'text', 't': 'YOLO'}]"
3070,k5sfhe,PLTR - Actual Solid DD-,https://www.reddit.com/r/wallstreetbets/commen...,tellit11,1,2020-12-03 07:01:02,PLTR is about to be the world's biggest peanut...,"[{'e': 'text', 't': 'DD'}]"
3071,k5udo0,Some autistic DD from the land down under,https://www.reddit.com/r/wallstreetbets/commen...,RioIsAGod,1,2020-12-03 09:52:22,You didn’t ask for it... but here it is! \n\nD...,[]
3072,k5xzb7,What is your go to source for doing DD on a co...,https://www.reddit.com/r/wallstreetbets/commen...,figgepop,1,2020-12-03 14:17:04,"As the title says, where do you go to do an in...","[{'e': 'text', 't': 'Discussion'}]"


In [ ]:
# WSB Filter by Post Length
wsbMaster2 = wsbMaster2.dropna(subset = ["Selftext"])
wsbMaster3 = wsbMaster2[wsbMaster2["Selftext"].str.split().apply(len) > 100].reset_index(drop = True)
wsbFinal = wsbMaster3.replace(r'\n',' ', regex=True)

In [ ]:
wsbFinal

,Post ID,Title,Url,Author,Score,Publish Date,Selftext,Flair
0,ej1m7j,Looking for DD on Australian firms or internat...,https://www.reddit.com/r/wallstreetbets/commen...,SFWonlyInvest,5,2020-01-02 18:00:00,"So the fires, how do we long this. Or to make ...",[]
1,ejnlo3,$CRC DD,https://www.reddit.com/r/wallstreetbets/commen...,notmichaelou,23,2020-01-03 23:18:04,"Alright autists, keeping up with the Daddy Ira...","[{'e': 'text', 't': 'DD'}]"
2,el6xlm,Beyond Air XAIR DD,https://www.reddit.com/r/wallstreetbets/commen...,jep777,1,2020-01-07 05:23:21,u/DrUNC83 mentioned this company in the daily ...,"[{'e': 'text', 't': 'DD'}]"
3,elg6sh,TSLA will dip tomorrow (DD inside),https://www.reddit.com/r/wallstreetbets/commen...,misolini,1,2020-01-07 19:18:26,"In Norway, we love electric cars. Due to our ...",[]
4,emoes4,New type of DD: Life Experience,https://www.reddit.com/r/wallstreetbets/commen...,BigBloodyShark,1,2020-01-10 09:00:30,This isn’t a shit post this is serious Just ...,"[{'e': 'text', 't': 'Discussion'}]"
...,...,...,...,...,...,...,...,...
979,k5kqaw,I just want to say thanks to the guy who poste...,https://www.reddit.com/r/wallstreetbets/commen...,happyidiot09,1,2020-12-02 23:03:18,Hopefully a few others got in on this play as ...,"[{'e': 'text', 't': 'Storytime'}]"
980,k5l9er,$COTY DD - Get rich with Kylie and Kim.,https://www.reddit.com/r/wallstreetbets/commen...,HyperVVenom,1,2020-12-02 23:31:50,Good day my fellow tards. You ever get jealous...,"[{'e': 'text', 't': 'Fundamentals'}]"
981,k5qudl,DBX DD,https://www.reddit.com/r/wallstreetbets/commen...,DOCTIME1,1,2020-12-03 05:03:11,"Listen up retards, I know what it seems like. ...","[{'e': 'text', 't': 'DD'}]"
982,k5sfhe,PLTR - Actual Solid DD-,https://www.reddit.com/r/wallstreetbets/commen...,tellit11,1,2020-12-03 07:01:02,PLTR is about to be the world's biggest peanut...,"[{'e': 'text', 't': 'DD'}]"


In [ ]:
stocksMaster.head(5)

,subreddit_name,post_created_utc,id,selftext,is_original_content,is_self,link_flair_text,locked,num_comments,over_18,score,spoiler,stickied,title,upvote_ratio,url
0,stocks,1.669889e+09,z9k0jl,Please use this thread to discuss your portfol...,False,True,NaN,False,589,False,152,False,True,Rate My Portfolio - r/Stocks Quarterly Thread ...,0.98,https://www.reddit.com/r/stocks/comments/z9k0j...
1,stocks,1.674556e+09,10k2i4m,"This is the daily discussion, so anything stoc...",False,True,NaN,False,351,False,25,False,True,r/Stocks Daily Discussion & Technicals Tuesday...,0.94,https://www.reddit.com/r/stocks/comments/10k2i...
2,stocks,1.674595e+09,10kgcjm,Microsoft shares rose in extended trading on T...,False,True,Company News,False,189,False,468,False,False,Microsoft shares rise on better-than-expected ...,0.94,https://www.reddit.com/r/stocks/comments/10kgc...
3,stocks,1.674591e+09,10kequb,https://www.cnbc.com/2023/01/23/google-ceo-def...,False,True,NaN,False,213,False,480,False,False,Google CEO defends job cuts in animated town h...,0.97,https://www.reddit.com/r/stocks/comments/10keq...
4,stocks,1.674572e+09,10k7ci7,https://www.cnbc.com/2023/01/24/many-new-york-...,False,True,NaN,False,248,False,893,False,False,NYSE says trading issue that led to dozens of ...,0.96,https://www.reddit.com/r/stocks/comments/10k7c...


In [ ]:
# Filter Useless： A Question is why we drop Dialy/Discussion Quaeterlt Thread, Quarterly Thread, Company News,Industry News
stocksMaster = stocksMaster.dropna(subset = ["selftext"])
dropS2Idx = (stocksMaster.loc[stocksMaster['title'].str.contains("Daily Discussion", case=False)]).index
dropS3Idx = (stocksMaster.loc[stocksMaster['title'].str.contains("Quarterly Thread", case=False)]).index
dropS4Idx = (stocksMaster.loc[stocksMaster['link_flair_text'].str.contains("Company News", case=False, na = False)]).index
dropS5Idx = (stocksMaster.loc[stocksMaster['link_flair_text'].str.contains("Industry News", case=False, na = False)]).index


investingMaster = investingMaster.dropna(subset = ["selftext"])
dropI2Idx = (investingMaster.loc[investingMaster['title'].str.contains("Thread", case=False)]).index

SMmaster = SMmaster.dropna(subset = ["selftext"])
dropSM1Idx = (SMmaster.loc[SMmaster['title'].str.contains("Daily Trading Report", case=False)]).index
dropSMFlair2Idx = (SMmaster.loc[SMmaster['link_flair_text'].str.contains("Newbie", case=False)]).index
dropSMFlair3Idx = (SMmaster.loc[SMmaster['link_flair_text'].str.contains("News", case=False)]).index
dropSMFlair4Idx = (SMmaster.loc[SMmaster['link_flair_text'].str.contains("Meme", case=False)]).index
# Newbie" serves a specific purpose. It is primarily used to indicate that the content or the questions 
# in the post are from users who are new to investing. 

# Dropping
stocksMaster2 = stocksMaster.drop(index = list(dropS2Idx))
stocksMaster2 = stocksMaster2.drop(index = list(dropS3Idx))
stocksMaster2 = stocksMaster2.drop(index = list(dropS4Idx))
stocksMaster2 = stocksMaster2.drop(index = list(dropS5Idx))

investingMaster2 = investingMaster.drop(index = list(dropI2Idx))

SMmaster2 = SMmaster.drop(index = list(dropSM1Idx))
SMmaster2 = SMmaster2.drop(index = list(dropSMFlair2Idx))
SMmaster2 = SMmaster2.drop(index = list(dropSMFlair3Idx))
SMmaster2 = SMmaster2.drop(index = list(dropSMFlair4Idx))

# Filter Length
stocksMaster3 = stocksMaster2[stocksMaster2["selftext"].str.split().apply(len) > 100].reset_index(drop = True)
investingMaster3 = investingMaster2[investingMaster2["selftext"].str.split().apply(len) > 100].reset_index(drop = True)
SMmaster3 = SMmaster2[SMmaster2["selftext"].str.split().apply(len) > 100].reset_index(drop = True)



stocksFinal = stocksMaster3.replace(r'\n',' ', regex=True)
investingFinal = investingMaster3.replace(r'\n',' ', regex=True)
smFinal =SMmaster3.replace(r'\n',' ', regex=True)

In [ ]:
print("Stocks: " + str(len(stocksFinal.index)))
print("WSB: " + str(len(wsbFinal.index)))
print("Stock Market: " + str(len(smFinal.index)))
print("Investing: " + str(len(investingFinal.index)))

Stocks: 419
WSB: 984
Stock Market: 142
Investing: 420


In [ ]:
wsbFinal.drop(index = 4, inplace = True)

In [ ]:
wsbFinal

,Post ID,Title,Url,Author,Score,Publish Date,Selftext,Flair
0,ej1m7j,Looking for DD on Australian firms or internat...,https://www.reddit.com/r/wallstreetbets/commen...,SFWonlyInvest,5,2020-01-02 18:00:00,"So the fires, how do we long this. Or to make ...",[]
1,ejnlo3,$CRC DD,https://www.reddit.com/r/wallstreetbets/commen...,notmichaelou,23,2020-01-03 23:18:04,"Alright autists, keeping up with the Daddy Ira...","[{'e': 'text', 't': 'DD'}]"
2,el6xlm,Beyond Air XAIR DD,https://www.reddit.com/r/wallstreetbets/commen...,jep777,1,2020-01-07 05:23:21,u/DrUNC83 mentioned this company in the daily ...,"[{'e': 'text', 't': 'DD'}]"
3,elg6sh,TSLA will dip tomorrow (DD inside),https://www.reddit.com/r/wallstreetbets/commen...,misolini,1,2020-01-07 19:18:26,"In Norway, we love electric cars. Due to our ...",[]
5,en8n1q,"Bulls, bears, idgaf who you are... listen up. ...",https://www.reddit.com/r/wallstreetbets/commen...,Kristoloy,1,2020-01-11 15:21:19,Damn. [$bynd](https://i.imgur.com/pwNQhXk.png)...,[]
...,...,...,...,...,...,...,...,...
979,k5kqaw,I just want to say thanks to the guy who poste...,https://www.reddit.com/r/wallstreetbets/commen...,happyidiot09,1,2020-12-02 23:03:18,Hopefully a few others got in on this play as ...,"[{'e': 'text', 't': 'Storytime'}]"
980,k5l9er,$COTY DD - Get rich with Kylie and Kim.,https://www.reddit.com/r/wallstreetbets/commen...,HyperVVenom,1,2020-12-02 23:31:50,Good day my fellow tards. You ever get jealous...,"[{'e': 'text', 't': 'Fundamentals'}]"
981,k5qudl,DBX DD,https://www.reddit.com/r/wallstreetbets/commen...,DOCTIME1,1,2020-12-03 05:03:11,"Listen up retards, I know what it seems like. ...","[{'e': 'text', 't': 'DD'}]"
982,k5sfhe,PLTR - Actual Solid DD-,https://www.reddit.com/r/wallstreetbets/commen...,tellit11,1,2020-12-03 07:01:02,PLTR is about to be the world's biggest peanut...,"[{'e': 'text', 't': 'DD'}]"


In [ ]:
wsbFinal.to_csv("wsbFinal.csv", index=False)
smFinal.to_csv("smFinal.csv", index=False)
stocksFinal.to_csv("stocksFinal.csv",index=False)
investingFinal.to_csv("investingFinal.csv",index=False)


In [ ]:
wsbFinal = pd.read_csv("example_posts (1).csv")
smFinal = pd.read_csv("example_posts (1).csv")
stocksbFinal = pd.read_csv("example_posts (1).csv")
investingbFinal = pd.read_csv("example_posts (1).csv")

In [ ]:
# Testing cell before creating a method -------------------------------------

testingData = wsbFinal["Selftext"][1]
# Removing junk string formatting characters \n and stuff
list1 = testingData.split(" ")
noN = []
for x in list1:
    noN.append(x.replace("\n", " "))

noStringFormattingLowercase = " ".join(noN).lower()

#print(noStringFormattingLowercase)

# punctuation
punct = string.punctuation
# keeping punctuation we need
punct = punct.replace("/", "")
punct = punct.replace("$", "")
punct = punct.replace("%", "")

noPunctLowerData = ''
for letter in noStringFormattingLowercase:
    if letter not in punct:
        noPunctLowerData += letter

#print("1 ", noPunctLowerData)

# tokenize, using regular expressions to keep special symbols
tokenizedLowerCaseCleanedData = regexp_tokenize(noPunctLowerData, pattern=r"\s|[\.,;']", gaps=True)

#print("2 ", tokenizedLowerCaseCleanedData)

from nltk.corpus import stopwords
# stop words
stopwords = stopwords.words('english')
# removing certain stop words
stopwords.remove("down")

# list of stop words we're adding
lst = ["however", "rightfully", "ill", "company", "mainly", "around", "has", "been", "a", "lot", "ive", "theyre", "eat", "food", "market"]
for i in lst:
  stopwords.append(i)

# removing stop words
tokenizedLowerCaseCleanedDataNoStop = []
for token in tokenizedLowerCaseCleanedData:
    if token not in stopwords:
        tokenizedLowerCaseCleanedDataNoStop.append(token)

tokenizedLowerCaseCleanedDataNoStop = " ".join(tokenizedLowerCaseCleanedDataNoStop)
#print("4 ", tokenizedLowerCaseCleanedDataNoStop)


#********************************************************************
# vectorizing
tfidf = TfidfVectorizer(stop_words = stopwords, min_df = 0.3)
corpus_tf = tfidf.fit_transform([tokenizedLowerCaseCleanedDataNoStop])
feats = tfidf.get_feature_names_out()

corpus_array = corpus_tf.toarray()

# dataframe sorted by descending
df = pd.DataFrame(corpus_array, columns = feats).T.sort_values(by = 0, ascending = False)
df


,0
crc,0.326802
debt,0.307579
oil,0.249908
year,0.153789
back,0.153789
...,...
1000,0.019224
bitch,0.019224
blow,0.019224
largest,0.019224


In [ ]:
def preprocessing(arrayOfData):
  outputArray = []
  tokenCount = []
  typeCount = []

  for document in arrayOfData:

    # Removing junk string formatting characters \n and stuff
    splitDocument = document.split(" ")
    noStringFormat = []
    for x in splitDocument:
        noStringFormat.append(x.replace("\n", " "))

    noStringFormattingLowercase = " ".join(noStringFormat).lower()

    #print(noStringFormattingLowercase)


    # punctuation
    punct = string.punctuation
    # keeping punctuation we need
    punct = punct.replace("/", "")
    punct = punct.replace("$", "")
    punct = punct.replace("%", "")

    noPunctLowerData = ''
    for letter in noStringFormattingLowercase:
        if letter not in punct:
            noPunctLowerData += letter

    #print("1 ", noPunctLowerData)

    # tokenize, using regular expressions to keep special symbols
    tokenizedLowerCaseCleanedData = regexp_tokenize(noPunctLowerData, pattern=r"\s|[\.,;']", gaps=True)

    #print("2 ", tokenizedLowerCaseCleanedData)

    from nltk.corpus import stopwords
    # stop words
    stopwords = stopwords.words('english')
    # removing certain stop words
    stopwords.remove("down")

    # list of stop words we're adding
    lst = ["however", "rightfully", "ill", "company", "mainly", "around", "has", "been", "a", "lot", "ive", "theyre", "eat", "food", "market", "would", "im", "us", "like", "years", "year", "also"
    , "could", "get", "https", "previewreddit", "one", "2022","2023", "iimgucom", "click", "x200b", "iimgurcom", "2020", "dont", "have", "was", "wa", "ha"]
    for i in lst:
      stopwords.append(i)

    #lemmmatizer = WordNetLemmatizer()
    # removing stop words
    import re
    tokenizedLowerCaseCleanedDataNoStop = []
    for token in tokenizedLowerCaseCleanedData:
        token = re.sub('[\W_]+', '', token)
        #token = lemmmatizer.lemmatize(token.lower())
        if token.strip() not in stopwords:
            tokenizedLowerCaseCleanedDataNoStop.append(token)


    tokenCount.append(len(tokenizedLowerCaseCleanedDataNoStop))
    typeCount.append(len(set(tokenizedLowerCaseCleanedDataNoStop)))

    tokenizedLowerCaseCleanedDataNoStop = " ".join(tokenizedLowerCaseCleanedDataNoStop)
    #print("4 ", tokenizedLowerCaseCleanedDataNoStop)

    outputArray.append(tokenizedLowerCaseCleanedDataNoStop)

  return [outputArray, tokenCount, typeCount]


In [ ]:
# Processing for TF-IDF Table

# Processing Investing Data
preprocessedDataInvesting = preprocessing(investingFinal["selftext"])
stringInvesting = " ".join(preprocessedDataInvesting[0])

# Processing Stock Market Data
preprocessedDataStockmarket = preprocessing(smFinal["selftext"])
stringStockmarket = " ".join(preprocessedDataStockmarket[0])

# Processing Stocks Data
preprocessedDataStocks = preprocessing(stocksFinal["selftext"])
stringStocks = " ".join(preprocessedDataStocks[0])

# Processing WSB Data
preprocessedDataWSB = preprocessing(wsbFinal["Selftext"])
stringWSB = " ".join(preprocessedDataWSB[0])

# vectorizing
tfidfVectorizer = TfidfVectorizer(stop_words = stopwords, min_df = 0.3)
corpus_tf = tfidfVectorizer.fit_transform([stringInvesting, stringStockmarket, stringStocks, stringWSB])
feats = tfidfVectorizer.get_feature_names_out()

corpus_array = corpus_tf.toarray()

# dataframe sorted by descending
tfidfTable = pd.DataFrame(corpus_array, columns = feats).T.sort_values(by = 0, ascending = False)
tfidfTable.columns = ["Investing", "Stock Market", "Stocks", "WSB"]
tfidfTable.head(10)

,Investing,Stock Market,Stocks,WSB
money,0.189952,0.064193,0.071503,0.127995
stock,0.151818,0.194956,0.338349,0.178120
time,0.137428,0.074892,0.136740,0.129660
price,0.129513,0.372081,0.117574,0.134284
stocks,0.128794,0.133141,0.157380,0.049385
know,0.118720,0.045173,0.053811,0.110238
funds,0.117281,0.010699,0.031329,0.011653
fund,0.108647,0.013076,0.024326,0.008508
rate,0.107208,0.073703,0.090300,0.037178
buy,0.107208,0.080836,0.071134,0.123001


In [ ]:
# Top TF-IDF Terms For each

# Investing top TF-IDF Terms
pd.DataFrame(tfidfTable["Investing"]).head(10)

,Investing
money,0.189952
stock,0.151818
time,0.137428
price,0.129513
stocks,0.128794
know,0.118720
funds,0.117281
fund,0.108647
rate,0.107208
buy,0.107208


In [ ]:
# Stock Market top TF-IDF Terms
pd.DataFrame(tfidfTable["Stock Market"]).sort_values(by=["Stock Market"], ascending = False).head(10)

,Stock Market
price,0.372081
stock,0.194956
support,0.139085
stocks,0.133141
major,0.133141
shares,0.115310
level,0.114121
new,0.112932
may,0.112932
down,0.109366


In [ ]:
# Stocks top TF-IDF Terms
pd.DataFrame(tfidfTable["Stocks"]).sort_values(by=["Stocks"], ascending = False).head(10)

,Stocks
stock,0.338349
earnings,0.168437
stocks,0.157380
time,0.136740
down,0.121629
price,0.117574
trading,0.113520
sp,0.107623
growth,0.106886
shares,0.101726


In [ ]:
# WSB top TF-IDF Terms
pd.DataFrame(tfidfTable["WSB"]).sort_values(by=["WSB"], ascending = False).head(10)

,WSB
going,0.188663
stock,0.178120
people,0.165173
earnings,0.136503
price,0.134284
time,0.129660
money,0.127995
buy,0.123001
even,0.122631
see,0.117452


In [ ]:
# Calculate Type to Token Ratio
def typeTokenRatio(inputArray):
  TTR = sum(inputArray[2]) / sum(inputArray[1])
  medianTokens = np.median(inputArray[1])
  avgTypes = sum(inputArray[2]) / len(inputArray[2])
  avgTokens = sum(inputArray[1]) / len(inputArray[1])

  return [TTR, medianTokens, avgTokens, avgTypes]

In [ ]:
# TTR Formatting
investingTTR = typeTokenRatio(preprocessedDataInvesting)
stockMarketTTR = typeTokenRatio(preprocessedDataStockmarket)
stocksTTR = typeTokenRatio(preprocessedDataStocks)
wsbTTR = typeTokenRatio(preprocessedDataWSB)

TTRdf = pd.DataFrame([investingTTR, stockMarketTTR, stocksTTR, wsbTTR])
TTRdf.columns = ["TTR", "Median Tokens", "Average Tokens", "Average Types"]
TTRdf.index = ["Investing", "Stock Market", "Stocks", "WSB"]

TTRdf.head()

,TTR,Median Tokens,Average Tokens,Average Types
Investing,0.742499,79.0,117.290476,87.088095
Stock Market,0.679703,141.5,207.795775,141.239437
Stocks,0.644798,96.0,232.198091,149.720764
WSB,0.716166,139.0,224.481707,160.766260


In [ ]:
# FRE Scores

def FREScore(inputArray):
  tokenized = inputArray[0]
  FREperDocumentScore = []
  for document in tokenized:
    FREperDocumentScore.append(textstat.flesch_reading_ease(document))

  return sum(FREperDocumentScore)/ len(inputArray[0])


In [ ]:
investingFRE = FREScore(preprocessedDataInvesting)
stockMarketFRE = FREScore(preprocessedDataStockmarket)
stocksFRE = FREScore(preprocessedDataStocks)
wsbFRE = FREScore(preprocessedDataWSB)

avgFREScore = [investingFRE, stockMarketFRE, stocksFRE, wsbFRE]

avgDf = TTRdf.copy()

avgDf["Average FRE Score"] = avgFREScore
avgDf


,TTR,Median Tokens,Average Tokens,Average Types,Average FRE Score
Investing,0.742499,79.0,117.290476,87.088095,-65.090095
Stock Market,0.679703,141.5,207.795775,141.239437,-166.495493
Stocks,0.644798,96.0,232.198091,149.720764,-183.420716
WSB,0.716166,139.0,224.481707,160.766260,-177.635833


In [ ]:
# MTLD

def mtldCalc(word_array, ttr_threshold):
    current_ttr = 1.0
    token_count = 0
    type_count = 0
    types = set()
    factors = 0.0

    for token in word_array:
        token_count += 1
        if token not in types:
            type_count +=1
            types.add(token)
        current_ttr = type_count / token_count
        if current_ttr <= ttr_threshold:
            factors += 1
            token_count = 0
            type_count = 0
            types = set()
            current_ttr = 1.0

    excess = 1.0 - current_ttr
    excess_val = 1.0 - ttr_threshold
    factors += excess / excess_val
    if factors != 0:
        return len(word_array) / factors
    return -1

#print(mtldCalc(stringInvesting.split(" "), 0.75))

def mtldHelper(preprocessedData):
  threshold = 0.75
  mtldPerDoc = []
  for doc in preprocessedData[0]:
    mtldPerDoc.append(mtldCalc(doc.split(" "), threshold))
  return mtldPerDoc

investingMTLD = np.mean(mtldHelper(preprocessedDataInvesting))
stockMarketMTLD = np.mean(mtldHelper(preprocessedDataStockmarket))
stocksMTLD = np.mean(mtldHelper(preprocessedDataStocks))
wsbMTLD = np.mean(mtldHelper(preprocessedDataWSB))

avgMTLDScore = [investingMTLD, stockMarketMTLD, stocksMTLD, wsbMTLD]

avgDf["Average MTLD Score"] = avgMTLDScore
avgDf


,TTR,Median Tokens,Average Tokens,Average Types,Average FRE Score,Average MTLD Score
Investing,0.742499,79.0,117.290476,87.088095,-65.090095,138.997814
Stock Market,0.679703,141.5,207.795775,141.239437,-166.495493,140.455765
Stocks,0.644798,96.0,232.198091,149.720764,-183.420716,139.540809
WSB,0.716166,139.0,224.481707,160.766260,-177.635833,197.657109


In [ ]:
def countVectFunc(inputArray):
  cv = CountVectorizer(min_df = 0.2, ngram_range = (1,2))
  return cv.fit_transform(inputArray)

cosineSimilarityArray = countVectFunc([stringInvesting, stringStockmarket, stringStocks, stringWSB])
pairwiseSim = cosine_similarity(cosineSimilarityArray)
cosineSimDf = pd.DataFrame(pairwiseSim)

cosineSimDf.index = ["Investing", "Stock Market", "Stocks", "WSB"]
cosineSimDf.columns = ["Investing", "Stock Market", "Stocks", "WSB"]

cosineSimDf
#stringInvesting
#stringStockmarket
#stringStocks
#stringWSB

,Investing,Stock Market,Stocks,WSB
Investing,1.000000,0.695363,0.796606,0.741579
Stock Market,0.695363,1.000000,0.756400,0.669896
Stocks,0.796606,0.756400,1.000000,0.763139
WSB,0.741579,0.669896,0.763139,1.000000


In [ ]:
positiveSentDict = {'grew': 1, 'profitable': 1, 'good' : 1, 'increase': 1, 'rally' : 1,
                    'strong': 1, 'surge' : 1, 'raise':1,"positive": 1, "growth":1, 'survive': 1,'up' : 1,'profit' : 1, "evolution":1,
                     "increased":1,  "recommend":1,  "high":1, "steady":1,  "strong":1,  "buy":1,  "green":1,  "bull":1,  "bullish":1}
negativeSentDict = {"low": -1, 'niche': -1, 'confused' : -1, 'discounting': -1, 'down': -1, 'cannot':-1, 'reduced': -1,
            'turbulent': -1, 'inflation' : -1, 'dropped': -1, 'bankruptcy': -1, 'crash': -1,
             "reduction":-1, 'volatility': -1,'bearish': -1,'dip' : -1,  "under":1}


sentDict = {**positiveSentDict, **negativeSentDict}
#sentDict = positiveSentDict | negativeSentDict


def SentScore(sentDict, docArray):
  sentScorePerDoc = []
  for document in docArray:
    sentScore = 0
    for token in document.split(" "):
        if token in sentDict.keys():
            #print(token, sentDict.get(token))
            sentScore += sentDict.get(token)
    normalizedScore = sentScore / len(document)
    sentScorePerDoc.append(normalizedScore)
  return sentScorePerDoc

investingAllSentScore = SentScore(sentDict, preprocessedDataInvesting[0])
StockmarketAllSentScore = SentScore(sentDict, preprocessedDataStockmarket[0])
StocksAllSentScore = SentScore(sentDict, preprocessedDataStocks[0])
WSBAllSentScore = SentScore(sentDict, preprocessedDataWSB[0])

sentScoreIndividualAllsubreddit = pd.DataFrame([investingAllSentScore, StockmarketAllSentScore, StocksAllSentScore, WSBAllSentScore]).T
sentScoreIndividualAllsubreddit.columns = ["Investing", "Stock Market", "Stocks", "WSB"]
sentScoreIndividualAllsubreddit

,Investing,Stock Market,Stocks,WSB
0,0.001969,0.000000,0.001542,0.000000
1,0.000000,0.004592,-0.001792,-0.001141
2,-0.008730,-0.003086,0.000307,0.001174
3,0.000000,0.000563,0.000000,0.000000
4,0.000000,0.000216,0.003404,0.006818
...,...,...,...,...
979,NaN,NaN,NaN,0.000000
980,NaN,NaN,NaN,0.001826
981,NaN,NaN,NaN,0.005260
982,NaN,NaN,NaN,0.002774


In [ ]:
avgDf["Average Sentiment"] = sentScoreIndividualAllsubreddit.mean(axis=0)
avgDf

,TTR,Median Tokens,Average Tokens,Average Types,Average FRE Score,Average MTLD Score,Average Sentiment
Investing,0.742499,79.0,117.290476,87.088095,-65.090095,138.997814,0.000940
Stock Market,0.679703,141.5,207.795775,141.239437,-166.495493,140.455765,0.000715
Stocks,0.644798,96.0,232.198091,149.720764,-183.420716,139.540809,0.000951
WSB,0.716166,139.0,224.481707,160.766260,-177.635833,197.657109,0.001236


In [1]:
stockTickerMaster = pd.read_excel("stockTicker.xlsx")

NameError: name 'pd' is not defined

In [ ]:
companyNames = list(stockTickerMaster.columns.values)[1:]
companyNames

['3M Company (MMM)',
 'Abbott Laboratories (ABT)',
 'AbbVie Inc. (ABBV)',
 'ABIOMED Inc (ABMD)',
 'Accenture plc (ACN)',
 'Activision Blizzard (ATVI)',
 'Adobe Inc. (ADBE)',
 'Advanced Micro Devices Inc (AMD)',
 'Advance Auto Parts (AAP)',
 'AES Corp (AES)',
 'AFLAC Inc (AFL)',
 'Agilent Technologies Inc (A)',
 'Air Products & Chemicals Inc (APD)',
 'Akamai Technologies Inc (AKAM)',
 'Alaska Air Group Inc (ALK)',
 'Albemarle Corp (ALB)',
 'Alexandria Real Estate Equities (ARE)',
 'Alexion Pharmaceuticals (ALXN)',
 'Align Technology (ALGN)',
 'Allegion (ALLE)',
 'Alliant Energy Corp (LNT)',
 'Allstate Corp (ALL)',
 'Alphabet Inc.\xa0(Class A) (GOOGL)',
 'Alphabet Inc.\xa0(Class C) (GOOG)',
 'Altria Group Inc (MO)',
 'Amazon.com Inc. (AMZN)',
 'Amcor plc (AMCR)',
 'Ameren Corp (AEE)',
 'American Airlines Group (AAL)',
 'American Electric Power (AEP)',
 'American Express Co (AXP)',
 'American International Group (AIG)',
 'American Tower Corp. (AMT)',
 'American Water Works Company Inc (AW

In [ ]:
longName = []
tickerLst = []
for i in companyNames:
  name = i.split("(")[0]
  name = name.strip()
  ticker = i.split("(")[1]
  ticker = ticker[:-1]
  longName.append(name)
  tickerLst.append(ticker)

In [ ]:
tickerLst

In [ ]:
tickerLst = map(lambda st: str.replace(st, "Class A)", "GOOG"), tickerLst)
tickerLst = list(map(lambda st: str.replace(st, "Class C)", "GOOGL"), tickerLst))
tickerLst

['MMM',
 'ABT',
 'ABBV',
 'ABMD',
 'ACN',
 'ATVI',
 'ADBE',
 'AMD',
 'AAP',
 'AES',
 'AFL',
 'A',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALXN',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOG',
 'GOOGL',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'ANTM',
 'AON',
 'AOS',
 'APA',
 'AIV',
 'AAPL',
 'AMAT',
 'APTV',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'BKR',
 'BLL',
 'BAC',
 'BK',
 'BAX',
 'BDX',
 'BRK-B',
 'BBY',
 'BIO',
 'BIIB',
 'BLK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BF-B',
 'CHRW',
 'COG',
 'CDNS',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'CNC',
 'CNP',
 'CTL',
 'CERN',
 'CF',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CTXS',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CMA',
 'CAG',
 'CXO'

In [ ]:
longName

['3M Company',
 'Abbott Laboratories',
 'AbbVie Inc.',
 'ABIOMED Inc',
 'Accenture plc',
 'Activision Blizzard',
 'Adobe Inc.',
 'Advanced Micro Devices Inc',
 'Advance Auto Parts',
 'AES Corp',
 'AFLAC Inc',
 'Agilent Technologies Inc',
 'Air Products & Chemicals Inc',
 'Akamai Technologies Inc',
 'Alaska Air Group Inc',
 'Albemarle Corp',
 'Alexandria Real Estate Equities',
 'Alexion Pharmaceuticals',
 'Align Technology',
 'Allegion',
 'Alliant Energy Corp',
 'Allstate Corp',
 'Alphabet Inc.',
 'Alphabet Inc.',
 'Altria Group Inc',
 'Amazon.com Inc.',
 'Amcor plc',
 'Ameren Corp',
 'American Airlines Group',
 'American Electric Power',
 'American Express Co',
 'American International Group',
 'American Tower Corp.',
 'American Water Works Company Inc',
 'Ameriprise Financial',
 'AmerisourceBergen Corp',
 'AMETEK Inc.',
 'Amgen Inc.',
 'Amphenol Corp',
 'Analog Devices, Inc.',
 'ANSYS',
 'Anthem',
 'Aon plc',
 'A.O. Smith Corp',
 'Apache Corporation',
 'Apartment Investment & Manageme

In [ ]:
#longName = list(map(lambda st: str.replace(st, " inc", ""), longName))
longName

['3M Company',
 'Abbott Laboratories',
 'AbbVie Inc.',
 'ABIOMED Inc',
 'Accenture plc',
 'Activision Blizzard',
 'Adobe Inc.',
 'Advanced Micro Devices Inc',
 'Advance Auto Parts',
 'AES Corp',
 'AFLAC Inc',
 'Agilent Technologies Inc',
 'Air Products & Chemicals Inc',
 'Akamai Technologies Inc',
 'Alaska Air Group Inc',
 'Albemarle Corp',
 'Alexandria Real Estate Equities',
 'Alexion Pharmaceuticals',
 'Align Technology',
 'Allegion',
 'Alliant Energy Corp',
 'Allstate Corp',
 'Alphabet Inc.',
 'Alphabet Inc.',
 'Altria Group Inc',
 'Amazon.com Inc.',
 'Amcor plc',
 'Ameren Corp',
 'American Airlines Group',
 'American Electric Power',
 'American Express Co',
 'American International Group',
 'American Tower Corp.',
 'American Water Works Company Inc',
 'Ameriprise Financial',
 'AmerisourceBergen Corp',
 'AMETEK Inc.',
 'Amgen Inc.',
 'Amphenol Corp',
 'Analog Devices, Inc.',
 'ANSYS',
 'Anthem',
 'Aon plc',
 'A.O. Smith Corp',
 'Apache Corporation',
 'Apartment Investment & Manageme

In [ ]:
tst2

'alright autists keeping daddy iran  mohamed bone saw theme giving dd crc inverse fuck crc california resource corporation largest oilgas producer california bitch spun oxy 2014 boy bad stripped account receivables left account payable gt debt basically fucked still doe result highest beta stock believe thehighest eampp sector peak post spin debt 7bn currently 5bn debt 1214bn ebitdax projected crc traditional steamflood producer emphasis crc doe utilize frackingwhich turn make well low declining 710 decline v 40 first decline see shale play whats deal yeah look bad post spin peak 90 oil ripping back low nearly 3 month ago right q3 earnings heavily shorted stock short interest 40 currently stock made run 2018 way 50 shit 2019 down long equity since 7 2017 trading option last fear mongering ability repay debt along ca batshit crazy legislative wise oil proscons saw thing run 7 50 think profiting volatility way isnt bad management team stellar todd stevens massive dick stated afraid play 

In [ ]:
# WSB Labelling
ticker = []
import textwrap

for row in wsbFinal.iterrows():
  title = row[1][1]
  body = row[1][6]

  titleWords = title.split(" ")
  bodyWords = body.split(" ")

  tmpTicker = []
  for word in titleWords:
    word = word.replace("$", "")
    if word in longName or word in tickerLst:
      if word == "DD":
        if "DuPont" in bodyWords or "DuPont" in titleWords:
          if word not in tmpTicker:
            tmpTicker.append(word)
      elif word == "A":
        if "Agilent" in bodyWords or "Agilent" in titleWords:
          if word not in tmpTicker:
            tmpTicker.append(word)
      else:
        if word not in tmpTicker:
            tmpTicker.append(word)
      #print("1: " + word)

  for word in bodyWords:
    word = word.replace("$", "")
    if word in longName or word in tickerLst:
      if word == "DD":
        if "DuPont" in bodyWords or "DuPont" in titleWords:
          if word not in tmpTicker:
            tmpTicker.append(word)
      elif word == "A":
        if "Agilent" in bodyWords or "Agilent" in titleWords:
          if word not in tmpTicker:
            tmpTicker.append(word)
      else:
        if word not in tmpTicker:
            tmpTicker.append(word)
      #print("2: " + word)

  if len(tmpTicker) == 0:
    tmpTicker.append("DELETE")

  ticker.append(tmpTicker)
#for i in wsbFinal["Selftext"]:
#  lst = tst2.split(" ")

#for word in lst:
#  if word in longName:
#    print("1: " + word)
#  if word in tickerLst:
#    print("2: " + word)


In [ ]:
ticker

In [ ]:
ticker2 = []
for i in ticker:
  if i[0] == "DELETE":
    s = ''.join(i)
    ticker2.append(s)
  else:
    ticker2.append(i)

ticker2

In [ ]:
ticker2 = []
for i in ticker:
  if len(i) == 1:
    s = ''.join(i)
    ticker2.append(s)
  else:
    ticker2.append(i)

ticker2

In [ ]:
wsbFinal["SentScore"] = WSBAllSentScore
wsbFinal["Tickers"] = ticker2

In [ ]:
wsbFinal = wsbFinal[wsbFinal["Tickers"] != "DELETE"].reset_index(drop=True)

In [ ]:
wsbFinal

,Post ID,Title,Url,Author,Score,Publish Date,Selftext,Flair,SentScore,Tickers
0,ejnlo3,$CRC DD,https://www.reddit.com/r/wallstreetbets/commen...,notmichaelou,23,2020-01-03 23:18:04,"Alright autists, keeping up with the Daddy Ira...","[{'e': 'text', 't': 'DD'}]",-0.001141,"[OXY, IR, HAL, SLB]"
1,eomcpm,AMZN DD INSIDE (LONG),https://www.reddit.com/r/wallstreetbets/commen...,TimCooksButtPlug,1,2020-01-14 15:00:29,**NOTE: This write up was done by u/soundofree...,"[{'e': 'text', 't': 'DD'}]",0.001189,"[AMZN, F]"
2,epc4op,Yet another $SPCE DD for all ye autists,https://www.reddit.com/r/wallstreetbets/commen...,achari01,1,2020-01-16 01:37:41,This is the ride of our generation. You didn’t...,"[{'e': 'text', 't': 'DD'}]",0.005563,"[AAPL, NFLX, AMZN, FB, GOOG]"
3,ermnes,FDX DD,https://www.reddit.com/r/wallstreetbets/commen...,notlehmanbros,1,2020-01-21 00:37:05,This afternoon I had my handler drive me to a ...,"[{'e': 'text', 't': 'DD'}]",0.000000,FDX
4,eshzd5,"[DD] Wuhan Coronavirus: $APT, $MMM",https://www.reddit.com/r/wallstreetbets/commen...,sheuv,1,2020-01-22 20:50:57,$APT makes protective equipment and tends to d...,[],0.001010,MMM
...,...,...,...,...,...,...,...,...,...,...
402,k4ojrd,DD: the biggest meme stock of all time gtfih (...,https://www.reddit.com/r/wallstreetbets/commen...,tooldrops,1,2020-12-01 16:44:12,"Sup gang. We have taken losses, and many of...","[{'e': 'text', 't': 'Fundamentals'}]",0.000000,"[HAS, IT, ALL]"
403,k4p5tg,Some Serious PLTR DD courtesy of SA,https://www.reddit.com/r/wallstreetbets/commen...,Namnikoo,1,2020-12-01 17:12:59,Just saw a great DD article on Seeking Alpha ...,"[{'e': 'text', 't': 'DD'}]",0.000048,"[SO, IT, PayPal]"
404,k4tw25,"$PSTH DD - Dumb Money has gone all in, and so ...",https://www.reddit.com/r/wallstreetbets/commen...,hooundabekeryda,1,2020-12-01 20:54:25,"For the past few weeks, there has been increas...","[{'e': 'text', 't': 'DD'}]",0.000600,EA
405,k5464b,From the Scientist who posted the NVAX DD: Upd...,https://www.reddit.com/r/wallstreetbets/commen...,Upstairs_Speech,1,2020-12-02 06:41:01,I've gotten numerous questions about what to m...,"[{'e': 'text', 't': 'DD'}]",0.001092,"[JNJ, JPM]"


In [ ]:
wsbFinal.to_csv("wsbFinal2.csv", index=False)

In [ ]:
wsbFinal.head(10)

,Post ID,Title,Url,Author,Score,Publish Date,Selftext,Flair
0,ej1m7j,Looking for DD on Australian firms or internat...,https://www.reddit.com/r/wallstreetbets/commen...,SFWonlyInvest,5,2020-01-02 18:00:00,"So the fires, how do we long this. Or to make ...",[]
1,ejnlo3,$CRC DD,https://www.reddit.com/r/wallstreetbets/commen...,notmichaelou,23,2020-01-03 23:18:04,"Alright autists, keeping up with the Daddy Ira...","[{'e': 'text', 't': 'DD'}]"
2,el6xlm,Beyond Air XAIR DD,https://www.reddit.com/r/wallstreetbets/commen...,jep777,1,2020-01-07 05:23:21,u/DrUNC83 mentioned this company in the daily ...,"[{'e': 'text', 't': 'DD'}]"
3,elg6sh,TSLA will dip tomorrow (DD inside),https://www.reddit.com/r/wallstreetbets/commen...,misolini,1,2020-01-07 19:18:26,"In Norway, we love electric cars. Due to our ...",[]
5,en8n1q,"Bulls, bears, idgaf who you are... listen up. ...",https://www.reddit.com/r/wallstreetbets/commen...,Kristoloy,1,2020-01-11 15:21:19,Damn. [$bynd](https://i.imgur.com/pwNQhXk.png)...,[]
...,...,...,...,...,...,...,...,...
979,k5kqaw,I just want to say thanks to the guy who poste...,https://www.reddit.com/r/wallstreetbets/commen...,happyidiot09,1,2020-12-02 23:03:18,Hopefully a few others got in on this play as ...,"[{'e': 'text', 't': 'Storytime'}]"
980,k5l9er,$COTY DD - Get rich with Kylie and Kim.,https://www.reddit.com/r/wallstreetbets/commen...,HyperVVenom,1,2020-12-02 23:31:50,Good day my fellow tards. You ever get jealous...,"[{'e': 'text', 't': 'Fundamentals'}]"
981,k5qudl,DBX DD,https://www.reddit.com/r/wallstreetbets/commen...,DOCTIME1,1,2020-12-03 05:03:11,"Listen up retards, I know what it seems like. ...","[{'e': 'text', 't': 'DD'}]"
982,k5sfhe,PLTR - Actual Solid DD-,https://www.reddit.com/r/wallstreetbets/commen...,tellit11,1,2020-12-03 07:01:02,PLTR is about to be the world's biggest peanut...,"[{'e': 'text', 't': 'DD'}]"


In [ ]:
smFinal

,subreddit_name,post_created_utc,id,selftext,is_original_content,is_self,link_flair_text,locked,num_comments,over_18,score,spoiler,stickied,title,upvote_ratio,url
0,StockMarket,1.674593e+09,10kfq5x,"**OVERALL:** A bit of a 'pause day,' at least ...",False,True,Technical Analysis,False,2,False,1,False,False,Technical Analysis & Trades: SPY QQQ IWM // JN...,1.00,https://www.reddit.com/r/StockMarket/comments/...
1,StockMarket,1.674593e+09,10kfmrx,ITM options have intrinsic value and a higher ...,False,True,Education/Lessons Learned,False,2,False,0,False,False,ITM vs. ATM Options,0.33,https://www.reddit.com/r/StockMarket/comments/...
2,StockMarket,1.674591e+09,10kexm2,"So I´m not a professional in stock trading, bu...",False,True,Help Needed,False,5,False,0,False,False,Stocks for School Project,0.50,https://www.reddit.com/r/StockMarket/comments/...
3,StockMarket,1.674586e+09,10kcl3m,Louisiana-based Lumen Technologies Inc ( Quant...,False,True,Fundamentals/DD,False,0,False,0,False,False,LUMN - Lumen Technologies value play,0.50,https://www.reddit.com/r/StockMarket/comments/...
4,StockMarket,1.674582e+09,10kay7u,"Hi everyone, **This isn't financial advice. P...",False,True,Fundamentals/DD,False,0,False,4,False,False,A small overview about the latest news around ...,1.00,https://www.reddit.com/r/StockMarket/comments/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,StockMarket,1.672978e+09,104k43z,1/5/2023 Market Tinfoil and Econ &#x200B; \-...,False,True,Fundamentals/DD,False,0,False,2,False,False,1/5/2023 Market Tinfoil and Econ,1.00,https://www.reddit.com/r/StockMarket/comments/...
138,StockMarket,1.672977e+09,104jusl,**OVERALL:** The bearish consolidation continu...,False,True,Technical Analysis,False,0,False,2,False,False,Daily Review & Trades: Technical Analysis of S...,0.75,https://www.reddit.com/r/StockMarket/comments/...
139,StockMarket,1.672972e+09,104i72l,The wealthy haven't bought stocks in over 50 y...,False,True,Education/Lessons Learned,False,16,False,0,False,False,The act of BUYING is only for retailers,0.37,https://www.reddit.com/r/StockMarket/comments/...
140,StockMarket,1.672959e+09,104cv46,Are you an experienced biotech professional wi...,False,True,Fundamentals/DD,False,0,False,0,False,False,Biotechnorati - a place for like-minded biotec...,0.50,https://www.reddit.com/r/StockMarket/comments/...


In [ ]:
# Other Labelling
ticker = []
import textwrap

for row in smFinal.iterrows():
  title = row[1][13]
  body = row[1][3]

  titleWords = title.split(" ")
  bodyWords = body.split(" ")

  tmpTicker = []
  for word in titleWords:
    word = word.replace("$", "")
    if word in longName or word in tickerLst:
      if word == "DD":
        if "DuPont" in bodyWords or "DuPont" in titleWords:
          if word not in tmpTicker:
            tmpTicker.append(word)
      elif word == "A":
        if "Agilent" in bodyWords or "Agilent" in titleWords:
          if word not in tmpTicker:
            tmpTicker.append(word)
      else:
        if word not in tmpTicker:
            tmpTicker.append(word)
      #print("1: " + word)

  for word in bodyWords:
    word = word.replace("$", "")
    if word in longName or word in tickerLst:
      if word == "DD":
        if "DuPont" in bodyWords or "DuPont" in titleWords:
          if word not in tmpTicker:
            tmpTicker.append(word)
      elif word == "A":
        if "Agilent" in bodyWords or "Agilent" in titleWords:
          if word not in tmpTicker:
            tmpTicker.append(word)
      else:
        if word not in tmpTicker:
            tmpTicker.append(word)
      #print("2: " + word)

  if len(tmpTicker) == 0:
    tmpTicker.append("DELETE")

  ticker.append(tmpTicker)
#for i in wsbFinal["Selftext"]:
#  lst = tst2.split(" ")

#for word in lst:
#  if word in longName:
#    print("1: " + word)
#  if word in tickerLst:
#    print("2: " + word)

ticker2 = []
for i in ticker:
  if i[0] == "DELETE":
    s = ''.join(i)
    ticker2.append(s)
  else:
    ticker2.append(i)

ticker2


ticker2 = []
for i in ticker:
  if len(i) == 1:
    s = ''.join(i)
    ticker2.append(s)
  else:
    ticker2.append(i)

ticker2

smFinal["SentScore"] = StockmarketAllSentScore
smFinal["Tickers"] = ticker2

smFinal = smFinal[smFinal["Tickers"] != "DELETE"].reset_index(drop=True)
smFinal

In [ ]:
# Investing
ticker = []
import textwrap

for row in investingFinal.iterrows():
  title = row[1][13]
  body = row[1][3]

  titleWords = title.split(" ")
  bodyWords = body.split(" ")

  tmpTicker = []
  for word in titleWords:
    word = word.replace("$", "")
    if word in longName or word in tickerLst:
      if word == "DD":
        if "DuPont" in bodyWords or "DuPont" in titleWords:
          if word not in tmpTicker:
            tmpTicker.append(word)
      elif word == "A":
        if "Agilent" in bodyWords or "Agilent" in titleWords:
          if word not in tmpTicker:
            tmpTicker.append(word)
      else:
        if word not in tmpTicker:
            tmpTicker.append(word)
      #print("1: " + word)

  for word in bodyWords:
    word = word.replace("$", "")
    if word in longName or word in tickerLst:
      if word == "DD":
        if "DuPont" in bodyWords or "DuPont" in titleWords:
          if word not in tmpTicker:
            tmpTicker.append(word)
      elif word == "A":
        if "Agilent" in bodyWords or "Agilent" in titleWords:
          if word not in tmpTicker:
            tmpTicker.append(word)
      else:
        if word not in tmpTicker:
            tmpTicker.append(word)
      #print("2: " + word)

  if len(tmpTicker) == 0:
    tmpTicker.append("DELETE")

  ticker.append(tmpTicker)
#for i in wsbFinal["Selftext"]:
#  lst = tst2.split(" ")

#for word in lst:
#  if word in longName:
#    print("1: " + word)
#  if word in tickerLst:
#    print("2: " + word)


ticker2 = []
for i in ticker:
  if len(i) == 1:
    s = ''.join(i)
    ticker2.append(s)
  else:
    ticker2.append(i)

investingFinal["SentScore"] = investingAllSentScore
investingFinal["Tickers"] = ticker2

investingFinal = investingFinal[investingFinal["Tickers"] != "DELETE"].reset_index(drop=True)
investingFinal

,subreddit_name,post_created_utc,id,selftext,is_original_content,is_self,link_flair_text,locked,num_comments,over_18,score,spoiler,stickied,title,upvote_ratio,url,SentScore,Tickers
0,investing,1.674568e+09,10k5xh1,"The other day I had a C Share mutual fund, HGH...",False,True,NaN,False,32,False,11,False,False,Class C > Class A Mutual Fund Conversion,0.91,https://www.reddit.com/r/investing/comments/10...,-0.008850,C
1,investing,1.674593e+09,10kfhj2,I know many are anti-financial advisors here a...,False,True,NaN,False,34,False,1,False,False,Is my cousin doing the right thing by using a ...,0.55,https://www.reddit.com/r/investing/comments/10...,0.000000,"[APA, Equinix, BK, Weyerhaeuser]"
2,investing,1.674531e+09,10jvwkx,I just don’t get it. Markets are supposed to b...,False,True,NaN,False,135,False,0,False,False,Can someone help make sense WTF is happening i...,0.45,https://www.reddit.com/r/investing/comments/10...,-0.001661,J
3,investing,1.674327e+09,10hyf98,I would like to build a position in a semicond...,False,True,NaN,False,35,False,18,False,False,Valuing Semiconductor manufacturers and a look...,0.86,https://www.reddit.com/r/investing/comments/10...,0.006082,"[MU, INTC, AMD]"
4,investing,1.674202e+09,10graoq,It's come to my attention recently they quietl...,False,True,NaN,False,7,False,4,False,False,E*Trade offers fractional shares for select ETFs,0.76,https://www.reddit.com/r/investing/comments/10...,0.000000,E*Trade
5,investing,1.674103e+09,10fsg7f,***High Short Interest Stocks Have Sharply Out...,False,True,NaN,False,2,False,0,False,False,GS Chart of the Day -> Short Covering Led Mark...,0.22,https://www.reddit.com/r/investing/comments/10...,0.001302,GS
6,investing,1.674016e+09,10exsrd,***Following is a Summary of Goldman Sachs' 1/...,False,True,NaN,False,2,False,4,False,False,Goldman Sachs Global Markets -> Equity Implied...,0.70,https://www.reddit.com/r/investing/comments/10...,-0.002038,GS
7,investing,1.674005e+09,10ety8p,# JPM Recommending Put Spreads on ALLY & KMX t...,False,True,NaN,False,0,False,0,False,False,JPM Tactical Derivatives Strategy (Summary) ->...,0.50,https://www.reddit.com/r/investing/comments/10...,-0.000342,"[JPM, KMX]"
8,investing,1.673967e+09,10edwzx,"Hi everyone, Since a few days I have been loo...",False,True,NaN,False,3,False,0,False,False,Alternative ETF for European tax-resident,0.50,https://www.reddit.com/r/investing/comments/10...,0.003055,MSCI
9,investing,1.673885e+09,10digxw,I have some shares in ONEOK (ticker: OKE) from...,False,True,NaN,False,5,False,0,False,False,"Dividends paid as ""Return of Capital"": why/how?",0.25,https://www.reddit.com/r/investing/comments/10...,0.000000,ONEOK


In [ ]:
# Stocks
ticker = []
import textwrap

for row in stocksFinal.iterrows():
  title = row[1][13]
  body = row[1][3]

  titleWords = title.split(" ")
  bodyWords = body.split(" ")

  tmpTicker = []
  for word in titleWords:
    word = word.replace("$", "")
    if word in longName or word in tickerLst:
      if word == "DD":
        if "DuPont" in bodyWords or "DuPont" in titleWords:
          if word not in tmpTicker:
            tmpTicker.append(word)
      elif word == "A":
        if "Agilent" in bodyWords or "Agilent" in titleWords:
          if word not in tmpTicker:
            tmpTicker.append(word)
      else:
        if word not in tmpTicker:
            tmpTicker.append(word)
      #print("1: " + word)

  for word in bodyWords:
    word = word.replace("$", "")
    if word in longName or word in tickerLst:
      if word == "DD":
        if "DuPont" in bodyWords or "DuPont" in titleWords:
          if word not in tmpTicker:
            tmpTicker.append(word)
      elif word == "A":
        if "Agilent" in bodyWords or "Agilent" in titleWords:
          if word not in tmpTicker:
            tmpTicker.append(word)
      else:
        if word not in tmpTicker:
            tmpTicker.append(word)
      #print("2: " + word)

  if len(tmpTicker) == 0:
    tmpTicker.append("DELETE")

  ticker.append(tmpTicker)
#for i in wsbFinal["Selftext"]:
#  lst = tst2.split(" ")

#for word in lst:
#  if word in longName:
#    print("1: " + word)
#  if word in tickerLst:
#    print("2: " + word)


ticker2 = []
for i in ticker:
  if len(i) == 1:
    s = ''.join(i)
    ticker2.append(s)
  else:
    ticker2.append(i)

stocksFinal["SentScore"] = StocksAllSentScore
stocksFinal["Tickers"] = ticker2

stocksFinal = stocksFinal[stocksFinal["Tickers"] != "DELETE"].reset_index(drop=True)
stocksFinal

,subreddit_name,post_created_utc,id,selftext,is_original_content,is_self,link_flair_text,locked,num_comments,over_18,score,spoiler,stickied,title,upvote_ratio,url,SentScore,Tickers
0,stocks,1.674526e+09,10juhvh,Source: https://stratechery.com/2023/netflixs-...,False,True,NaN,False,37,False,49,False,False,"WBD has $50.4bn in debt, DIS has $45bn, PARA h...",0.79,https://www.reddit.com/r/stocks/comments/10juh...,0.003404,"[DIS, NFLX]"
1,stocks,1.674575e+09,10k8880,So I'm new to the world of investing. I hold a...,False,True,Advice Request,False,13,False,4,False,False,Should I diversify my Portfolio?,0.75,https://www.reddit.com/r/stocks/comments/10k88...,0.003413,CSCO
2,stocks,1.674567e+09,10k5itf,#Good morning traders and investors of the r/s...,False,True,NaN,False,1,False,4,False,False,(1/24) Tuesday's Pre-Market Stock Movers & News,0.70,https://www.reddit.com/r/stocks/comments/10k5i...,0.001408,"[VZ, JNJ, LMT, AMD]"
3,stocks,1.674411e+09,10iq0ld,Given Sundar Pichai awful track record ever si...,False,True,Opinion,False,508,False,1250,False,False,Sundar Pichai (Google's CEO) has to go and som...,0.80,https://www.reddit.com/r/stocks/comments/10iq0...,-0.002833,GOOGL
4,stocks,1.674522e+09,10jt5r7,"As always, below represents my opinions and sh...",False,True,Company Analysis,False,8,False,6,False,False,Credit Acceptance Corporation (CACC) Stock Rev...,0.87,https://www.reddit.com/r/stocks/comments/10jt5...,0.001259,IR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,stocks,1.671118e+09,zmnqdm,"As someone who is maxing 401k, Roth IRA and HS...",False,True,Advice Request,False,12,False,11,False,False,What goes into your taxable/non-taxable accounts?,0.80,https://www.reddit.com/r/stocks/comments/zmnqd...,0.011472,O
84,stocks,1.671113e+09,zmlouz,This macroenvironment has made a lot of CEOs h...,False,True,Industry Discussion,False,81,False,90,False,False,CEOs that are willing to put their money where...,0.81,https://www.reddit.com/r/stocks/comments/zmlou...,0.004622,"[PSA, D]"
85,stocks,1.671112e+09,zmlkpw,#Good morning traders and investors of the r/s...,False,True,NaN,False,0,False,8,False,False,(12/15) Thursday's Pre-Market Stock Movers & News,0.85,https://www.reddit.com/r/stocks/comments/zmlkp...,-0.000477,"[LEN, WDC, T]"
86,stocks,1.671052e+09,zm22xy,Hoping to get some help deciding what funds to...,False,True,Advice Request,False,3,False,6,False,False,Deciding What to Sell to Correct a Roth IRA Ov...,0.75,https://www.reddit.com/r/stocks/comments/zm22x...,0.000000,D


In [ ]:
smFinal.to_csv("smFinal2.csv", index=False)
stocksFinal.to_csv("stocksFinal2.csv", index=False)
investingFinal.to_csv("investingFinal2.csv", index=False)


------------- BERT analysis ---------------


In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 7.3 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import tensorflow as tf
import pandas as pd
URL = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file(fname="aclImdb_v1.tar.gz",
                                  origin=URL,
                                  untar=True,
                                  cache_dir='.',
                                  cache_subdir='')

84125825/84125825 [==============================] - 41s 0us/step


In [ ]:
# The shutil module offers a number of high-level
# operations on files and collections of files.
import os
import shutil
# Create main directory path ("/aclImdb")
main_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
# Create sub directory path ("/aclImdb/train")
train_dir = os.path.join(main_dir, 'train')
# Remove unsup folder since this is a supervised learning task
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)
# View the final train folder
print(os.listdir(train_dir))

['urls_pos.txt', 'neg', 'labeledBow.feat', 'urls_neg.txt', 'pos', 'urls_unsup.txt', 'unsupBow.feat']


In [ ]:
# We create a training dataset and a validation
# dataset from our "aclImdb/train" directory with a 80/20 split.
train = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=30000, validation_split=0.2,
    subset='training', seed=123)
test = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=30000, validation_split=0.2,
    subset='validation', seed=123)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [ ]:
for i in train.take(1):
  train_feat = i[0].numpy()
  train_lab = i[1].numpy()

train = pd.DataFrame([train_feat, train_lab]).T
train.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
train['DATA_COLUMN'] = train['DATA_COLUMN'].str.decode("utf-8")
train.head()

,DATA_COLUMN,LABEL_COLUMN
0,Canadian director Vincenzo Natali took the art...,1
1,I gave this film 10 not because it is a superb...,1
2,I admit to being somewhat jaded about the movi...,1
3,"For a long time, 'The Menagerie' was my favori...",1
4,A truly frightening film. Feels as if it were ...,0


In [ ]:
for j in test.take(1):
  test_feat = j[0].numpy()
  test_lab = j[1].numpy()

test = pd.DataFrame([test_feat, test_lab]).T
test.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
test['DATA_COLUMN'] = test['DATA_COLUMN'].str.decode("utf-8")
test.head()

,DATA_COLUMN,LABEL_COLUMN
0,I can't believe that so much talent can be was...,0
1,This movie blows - let's get that straight rig...,0
2,"The saddest thing about this ""tribute"" is that...",0
3,I'm only rating this film as a 3 out of pity b...,0
4,Something surprised me about this movie - it w...,1


In [ ]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN):
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN],
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN],
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  return train_InputExamples, validation_InputExamples

  train_InputExamples, validation_InputExamples = convert_data_to_examples(train,
                                                                           test,
                                                                           'DATA_COLUMN',
                                                                           'LABEL_COLUMN')

def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'DATA_COLUMN'
LABEL_COLUMN = 'LABEL_COLUMN'

In [ ]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data)

Epoch 1/2
     14/Unknown - 776s 52s/step - loss: 0.6990 - accuracy: 0.5067

In [ ]:
pred_sentences = ['This was an awesome movie. I watch it twice my time watching this beautiful movie if I have known it was this good',
                  'One of the worst movies of all time. I cannot believe I wasted two hours of my life for this movie']


In [ ]:
tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = ['Negative','Positive']
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()
for i in range(len(pred_sentences)):
  print(pred_sentences[i], ": \n", labels[label[i]])